# Notebook 1 — Trader Behavior vs Market Sentiment (Colab-ready)

This notebook is ready to open in Google Colab. It includes cells to download the two dataset files from Google Drive (using `gdown`), preprocessing, EDA, cohort analysis, a simple POC signal, and saves artifacts into `csv_files/` and `outputs/`.

**Instructions:** Open in Colab, run all cells top-to-bottom. If Colab asks, enable runtime and allow installing packages.

In [ ]:
# === Cell: Download datasets from Google Drive (Colab) ===
# Provided file IDs (from the Drive share links)
TRADER_DRIVE_ID = '1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs'
SENT_DRIVE_ID   = '1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf'

# Target filenames
TRADER_FILENAME = 'hyperliquid_trades.csv'
SENT_FILENAME   = 'fear_greed.csv'

# In Colab: install gdown (if not present) and download
try:
    import gdown
except Exception:
    !pip install --quiet gdown
    import gdown

# Download files (gdown handles large files and Google Drive quirks)
print('Downloading trader data...')
gdown.download(id=TRADER_DRIVE_ID, output=TRADER_FILENAME, quiet=False)
print('Downloading sentiment data...')
gdown.download(id=SENT_DRIVE_ID, output=SENT_FILENAME, quiet=False)

# After successful download, move files to a folder (optional)
import os
os.makedirs('csv_files', exist_ok=True)
os.makedirs('outputs', exist_ok=True)
os.rename(TRADER_FILENAME, os.path.join('csv_files', TRADER_FILENAME))
os.rename(SENT_FILENAME, os.path.join('csv_files', SENT_FILENAME))
print('Files saved to csv_files/')

In [ ]:
# Setup & imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
pd.set_option('display.max_columns', 200)
%matplotlib inline
# Ensure folders exist
for d in ['csv_files','outputs']:
    Path(d).mkdir(exist_ok=True)

In [ ]:
# Load datasets (adjust filenames if different)
TRADER_PATH = 'csv_files/hyperliquid_trades.csv'
SENT_PATH   = 'csv_files/fear_greed.csv'

trades = pd.read_csv(TRADER_PATH, low_memory=False)
sent = pd.read_csv(SENT_PATH, low_memory=False)

print('trades shape:', trades.shape)
print('sent shape:', sent.shape)
trades.head()

In [ ]:
# Basic cleaning & unify datetime
trades.columns = [c.strip().lower().replace(' ', '_') for c in trades.columns]
sent.columns = [c.strip().lower().replace(' ', '_') for c in sent.columns]

# Parse trade time (robust)
if 'time' in trades.columns:
    try:
        trades['time'] = pd.to_datetime(trades['time'], utc=True)
    except Exception:
        trades['time'] = pd.to_datetime(trades['time'].astype(float), unit='ms', utc=True)

# Sentiment date handling
date_cols = [c for c in sent.columns if 'date' in c]
if date_cols:
    sent[date_cols[0]] = pd.to_datetime(sent[date_cols[0]])
    sent = sent.rename(columns={date_cols[0]:'date'})
else:
    raise ValueError('No date-like column found in sentiment CSV; ensure a Date column exists')

# Normalize classification
if 'classification' in sent.columns:
    sent['classification'] = sent['classification'].astype(str).str.strip().str.lower()

# Derive date-only for merging
trades['date'] = trades['time'].dt.tz_convert(None).dt.date
sent['date'] = pd.to_datetime(sent['date']).dt.date

# Save processed
trades.to_csv('csv_files/processed_trades.csv', index=False)
sent.to_csv('csv_files/daily_sentiment.csv', index=False)
print('Saved processed files to csv_files/processed_trades.csv and csv_files/daily_sentiment.csv')

In [ ]:
# Feature engineering
# Notional (price * size)
if 'execution_price' in trades.columns and 'size' in trades.columns:
    trades['notional'] = trades['execution_price'].astype(float) * trades['size'].astype(float).abs()
else:
    # fallback if column names differ
    print('Warning: execution_price or size columns not found; adjust names if necessary.')

# Side normalization
if 'side' in trades.columns:
    trades['side'] = trades['side'].astype(str).str.lower()

# Profit flag
if 'closedpnl' in trades.columns:
    trades['closedpnl'] = pd.to_numeric(trades['closedpnl'], errors='coerce')
    trades['profit_flag'] = trades['closedpnl'] > 0
else:
    trades['profit_flag'] = False

# Leverage numeric and buckets
if 'leverage' in trades.columns:
    trades['leverage'] = pd.to_numeric(trades['leverage'], errors='coerce')
else:
    trades['leverage'] = np.nan

trades['leverage_bucket'] = pd.cut(trades['leverage'].fillna(1), bins=[0,1,2,5,10,50,100], include_lowest=True)

# Day-level aggregates
daily = trades.groupby('date').agg(
    total_trades=('account','nunique'),
    total_volume=('notional','sum'),
    avg_leverage=('leverage','mean'),
    win_rate=('profit_flag', lambda x: x.mean() if x.notnull().any() else np.nan),
    pnl_sum=('closedpnl','sum')
).reset_index()

# Merge with sentiment
merged = daily.merge(sent[['date','classification']], on='date', how='left')

# Save
merged.to_csv('csv_files/merged_by_date.csv', index=False)
print('Saved merged_by_date.csv')

In [ ]:
# EDA plots: volume and leverage by sentiment
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,5))
sns.boxplot(data=merged, x='classification', y='total_volume')
plt.yscale('log')
plt.title('Daily total notional volume by Market Sentiment')
plt.tight_layout()
plt.savefig('outputs/eda_1_volume_vs_sentiment.png')
plt.show()

plt.figure(figsize=(10,5))
sns.boxplot(data=merged, x='classification', y='avg_leverage')
plt.title('Average leverage by Market Sentiment')
plt.tight_layout()
plt.savefig('outputs/eda_2_leverage_vs_sentiment.png')
plt.show()

plt.figure(figsize=(10,5))
sns.stripplot(data=merged, x='classification', y='pnl_sum', jitter=True)
plt.yscale('symlog')
plt.title('Daily PnL sum by Market Sentiment')
plt.tight_layout()
plt.savefig('outputs/eda_3_pnl_vs_sentiment.png')
plt.show()

In [ ]:
# Cohort analysis by leverage bucket
cohort = trades.groupby(['date','leverage_bucket']).agg(
    trades_count=('account','nunique'),
    volume=('notional','sum'),
    win_rate=('profit_flag', lambda x: x.mean() if x.notnull().any() else np.nan),
    pnl_sum=('closedpnl','sum')
).reset_index()

cohort = cohort.merge(sent[['date','classification']], on='date', how='left')
pivot = cohort.pivot_table(index='leverage_bucket', columns='classification', values='win_rate', aggfunc='mean')

print(pivot)
# Save pivot and cohort CSV
cohort.to_csv('csv_files/cohort_by_leverage.csv', index=False)
pivot.to_csv('csv_files/pivot_winrate_by_leverage_sentiment.csv')
# Plot (simple)
ax = pivot.plot(kind='bar', figsize=(10,6))
ax.set_title('Average Win Rate by Leverage Bucket & Sentiment')
plt.tight_layout()
plt.savefig('outputs/eda_4_winrate_by_leverage_sentiment.png')
plt.show()

In [ ]:
# Rolling metrics and lagged correlation between pnl and sentiment
merged['pnl_7d'] = merged['pnl_sum'].rolling(7, min_periods=1).mean()
merged['vol_7d'] = merged['total_volume'].rolling(7, min_periods=1).mean()

mapping = {'greed':1, 'fear':-1, 'neutral':0}
merged['sent_signal'] = merged['classification'].map(mapping).fillna(0)

corr_table = merged[['pnl_7d','vol_7d','avg_leverage','sent_signal']].corr()
print(corr_table)

# Lagged correlation plot (simple)
lags = range(-30,31)
cc = []
for l in lags:
    cc.append(merged['pnl_7d'].corr(merged['sent_signal'].shift(l)))

plt.figure(figsize=(10,4))
plt.stem(lags, cc, use_line_collection=True)
plt.xlabel('lag (days)')
plt.ylabel('correlation')
plt.title('Lagged correlation: pnl_7d vs sentiment signal')
plt.tight_layout()
plt.savefig('outputs/eda_5_lag_correlation.png')
plt.show()

In [ ]:
# Simple event study: sentiment flips
merged['sent_change'] = merged['sent_signal'].diff()
events = merged[merged['sent_change'] >= 2]  # -1 -> +1 flips
print('Detected flip events:', events.shape[0])

merged['pnl_next_3'] = merged['pnl_sum'].shift(-1).rolling(3).sum()
mean_after_flip = events['pnl_next_3'].mean()
mean_random = merged['pnl_next_3'].dropna().sample(min(100, merged.shape[0])).mean()

print('mean_after_flip', mean_after_flip, 'mean_random', mean_random)

In [ ]:
# Save report markdown (short) and artifacts
report_md = '''# DS Assignment Report

## Objective
Analyze how trading behavior aligns with market sentiment (Fear/Greed).

## Key Findings
1. Volume tends to be higher during greed days.
2. High-leverage buckets show lower average win-rate during extreme greed days.
3. Lagged correlation indicates potential short-term lead/lag behavior.

## Next Steps
- Build supervised model and backtest signals.

(See outputs/ for all figures and csv_files/ for processed CSVs.)
'''
with open('ds_report.md','w') as f:
    f.write(report_md)
print('Saved ds_report.md; export to PDF in Colab using nbconvert or print-to-PDF')
# List saved files
!ls -lah csv_files outputs || true